<a href="https://colab.research.google.com/github/anshatwork/llm-search/blob/main/raptor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install numpy pandas umap-learn sentence-transformers scikit-learn torch langchain langchain-core langchain-community llama-cpp-python torch chromadb  pymilvus==2.4.3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metad

In [ ]:
!pip install PyPDF2 nltk matplotlib numpy scikit-learn sentence-transformers pandas umap-learn tiktoken llama-cpp-python


In [ ]:
!wget "https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q4_K_M.gguf"


In [ ]:
!pip install -U langchain umap-learn scikit-learn langchain_community tiktoken langchainhub chromadb
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1

In [ ]:
!pip uninstall pymilvus

Found existing installation: pymilvus 2.4.3
Uninstalling pymilvus-2.4.3:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/pymilvus-2.4.3.dist-info/*
    /usr/local/lib/python3.10/dist-packages/pymilvus/*
Proceed (Y/n)? y
  Successfully uninstalled pymilvus-2.4.3


In [ ]:
!pip install --upgrade torch sentence-transformers pymilvus==2.4.3 langchain


In [ ]:
from PyPDF2 import PdfReader
import nltk
from nltk.tokenize import word_tokenize

# Ensure the necessary resources are downloaded
nltk.download('punkt')

import base64

from langchain_community.vectorstores import Milvus
from langchain.schema import Document
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection
from sentence_transformers import SentenceTransformer
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sentence_transformers import SentenceTransformer
from langchain_community.llms.llamacpp import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate, HumanMessagePromptTemplate
from langchain import hub
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd
import umap
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sentence_transformers import SentenceTransformer
from langchain_community.llms.llamacpp import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate, HumanMessagePromptTemplate
from langchain import hub
from sklearn.mixture import GaussianMixture
from langchain_core.runnables import RunnablePassthrough
import torch


embd = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize the LlamaCpp model for summarization
n_gpu_layers = -1
n_batch = 512
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

RANDOM_SEED = 224  # Fixed seed for reproducibility

# Initialize LlamaCpp model
model = LlamaCpp(
    model_path="zephyr-7b-beta.Q4_K_M.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    temperature=0.75,
    max_tokens=1000,
    top_p=1,
    n_ctx=35000,
    callback_manager=callback_manager,
    verbose=True,
)

print("Initialized LlamaCpp model")

# Function definitions



RANDOM_SEED = 224  # Fixed seed for reproducibility


connections.connect(
    alias="default",
    uri="https://in03-a02578c01622c70.api.gcp-us-west1.zillizcloud.com",
    token="472773d1d6fe2bdf1be4b9281e9d663e86296fc897065eb2e3e8161b3928c0317fe3fabb0c9c3afa83051c34f68173d5536e77a7"
)

# Prepare documents with metadata
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
    FieldSchema(name="book_title", dtype=DataType.VARCHAR, max_length=255),
    FieldSchema(name="chunk", dtype=DataType.VARCHAR, max_length=10000)
]
schema = CollectionSchema(fields, "Textbook content with metadata")

# Create or load the collection
collection = Collection("stepsAI1", schema=schema)

def generateSummary(book_title,docs_texts):

    embd = SentenceTransformer("all-MiniLM-L6-v2")


    def global_cluster_embeddings(
        embeddings: np.ndarray,
        dim: int,
        n_neighbors: Optional[int] = None,
        metric: str = "cosine",
    ) -> np.ndarray:
        """
        Perform global dimensionality reduction on the embeddings using UMAP.

        Parameters:
        - embeddings: The input embeddings as a numpy array.
        - dim: The target dimensionality for the reduced space.
        - n_neighbors: Optional; the number of neighbors to consider for each point.
                      If not provided, it defaults to the square root of the number of embeddings.
        - metric: The distance metric to use for UMAP.

        Returns:
        - A numpy array of the embeddings reduced to the specified dimensionality.
        """
        if n_neighbors is None:
            n_neighbors = int((len(embeddings) - 1) ** 0.5)
        return umap.UMAP(
            n_neighbors=n_neighbors, n_components=dim, metric=metric
        ).fit_transform(embeddings)


    def local_cluster_embeddings(
        embeddings: np.ndarray, dim: int, num_neighbors: int = 10, metric: str = "cosine"
    ) -> np.ndarray:
        """
        Perform local dimensionality reduction on the embeddings using UMAP, typically after global clustering.

        Parameters:
        - embeddings: The input embeddings as a numpy array.
        - dim: The target dimensionality for the reduced space.
        - num_neighbors: The number of neighbors to consider for each point.
        - metric: The distance metric to use for UMAP.

        Returns:
        - A numpy array of the embeddings reduced to the specified dimensionality.
        """
        return umap.UMAP(
            n_neighbors=num_neighbors, n_components=dim, metric=metric
        ).fit_transform(embeddings)


    def get_optimal_clusters(
        embeddings: np.ndarray, max_clusters: int = 50, random_state: int = RANDOM_SEED
    ) -> int:
        """
        Determine the optimal number of clusters using the Bayesian Information Criterion (BIC) with a Gaussian Mixture Model.

        Parameters:
        - embeddings: The input embeddings as a numpy array.
        - max_clusters: The maximum number of clusters to consider.
        - random_state: Seed for reproducibility.

        Returns:
        - An integer representing the optimal number of clusters found.
        """
        max_clusters = min(max_clusters, len(embeddings))
        n_clusters = np.arange(1, max_clusters)
        bics = []
        for n in n_clusters:
            gm = GaussianMixture(n_components=n, random_state=random_state)
            gm.fit(embeddings)
            bics.append(gm.bic(embeddings))
        return n_clusters[np.argmin(bics)]


    def GMM_cluster(embeddings: np.ndarray, threshold: float, random_state: int = 0):
        """
        Cluster embeddings using a Gaussian Mixture Model (GMM) based on a probability threshold.

        Parameters:
        - embeddings: The input embeddings as a numpy array.
        - threshold: The probability threshold for assigning an embedding to a cluster.
        - random_state: Seed for reproducibility.

        Returns:
        - A tuple containing the cluster labels and the number of clusters determined.
        """
        n_clusters = get_optimal_clusters(embeddings)
        gm = GaussianMixture(n_components=n_clusters, random_state=random_state)
        gm.fit(embeddings)
        probs = gm.predict_proba(embeddings)
        labels = [np.where(prob > threshold)[0] for prob in probs]
        return labels, n_clusters


    def perform_clustering(
        embeddings: np.ndarray,
        dim: int,
        threshold: float,
    ) -> List[np.ndarray]:
        """
        Perform clustering on the embeddings by first reducing their dimensionality globally, then clustering
        using a Gaussian Mixture Model, and finally performing local clustering within each global cluster.

        Parameters:
        - embeddings: The input embeddings as a numpy array.
        - dim: The target dimensionality for UMAP reduction.
        - threshold: The probability threshold for assigning an embedding to a cluster in GMM.

        Returns:
        - A list of numpy arrays, where each array contains the cluster IDs for each embedding.
        """
        if len(embeddings) <= dim + 1:
            # Avoid clustering when there's insufficient data
            return [np.array([0]) for _ in range(len(embeddings))]

        # Global dimensionality reduction
        reduced_embeddings_global = global_cluster_embeddings(embeddings, dim)
        # Global clustering
        global_clusters, n_global_clusters = GMM_cluster(
            reduced_embeddings_global, threshold
        )

        all_local_clusters = [np.array([]) for _ in range(len(embeddings))]
        total_clusters = 0

        # Iterate through each global cluster to perform local clustering
        for i in range(n_global_clusters):
            # Extract embeddings belonging to the current global cluster
            global_cluster_embeddings_ = embeddings[
                np.array([i in gc for gc in global_clusters])
            ]

            if len(global_cluster_embeddings_) == 0:
                continue
            if len(global_cluster_embeddings_) <= dim + 1:
                # Handle small clusters with direct assignment
                local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
                n_local_clusters = 1
            else:
                # Local dimensionality reduction and clustering
                reduced_embeddings_local = local_cluster_embeddings(
                    global_cluster_embeddings_, dim
                )
                local_clusters, n_local_clusters = GMM_cluster(
                    reduced_embeddings_local, threshold
                )

            # Assign local cluster IDs, adjusting for total clusters already processed
            for j in range(n_local_clusters):
                local_cluster_embeddings_ = global_cluster_embeddings_[
                    np.array([j in lc for lc in local_clusters])
                ]
                indices = np.where(
                    (embeddings == local_cluster_embeddings_[:, None]).all(-1)
                )[1]
                for idx in indices:
                    all_local_clusters[idx] = np.append(
                        all_local_clusters[idx], j + total_clusters
                    )

            total_clusters += n_local_clusters

        return all_local_clusters


    ### --- Our code below --- ###


    def embed(texts):
        """
        Generate embeddings for a list of text documents.

        This function assumes the existence of an `embd` object with a method `embed_documents`
        that takes a list of texts and returns their embeddings.

        Parameters:
        - texts: List[str], a list of text documents to be embedded.

        Returns:
        - numpy.ndarray: An array of embeddings for the given text documents.
        """
        text_embeddings = embd.encode(texts)
        text_embeddings_np = np.array(text_embeddings)
        return text_embeddings_np


    def embed_cluster_texts(texts):
        """
        Embeds a list of texts and clusters them, returning a DataFrame with texts, their embeddings, and cluster labels.

        This function combines embedding generation and clustering into a single step. It assumes the existence
        of a previously defined `perform_clustering` function that performs clustering on the embeddings.

        Parameters:
        - texts: List[str], a list of text documents to be processed.

        Returns:
        - pandas.DataFrame: A DataFrame containing the original texts, their embeddings, and the assigned cluster labels.
        """
        text_embeddings_np = embed(texts)  # Generate embeddings
        cluster_labels = perform_clustering(
            text_embeddings_np, 10, 0.1
        )  # Perform clustering on the embeddings
        df = pd.DataFrame()  # Initialize a DataFrame to store the results
        df["text"] = texts  # Store original texts
        df["embd"] = list(text_embeddings_np)  # Store embeddings as a list in the DataFrame
        df["cluster"] = cluster_labels  # Store cluster labels
        return df


    def fmt_txt(df: pd.DataFrame) -> str:
        """
        Formats the text documents in a DataFrame into a single string.

        Parameters:
        - df: DataFrame containing the 'text' column with text documents to format.

        Returns:
        - A single string where all text documents are joined by a specific delimiter.
        """
        unique_txt = df["text"].tolist()
        unique_txt = [str(text) for text in unique_txt]  # Convert to string if necessary
        return "--- --- \n --- --- ".join(unique_txt)


    def embed_cluster_summarize_texts(
        texts: List[str], level: int
    ) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Embeds, clusters, and summarizes a list of texts. This function first generates embeddings for the texts,
        clusters them based on similarity, expands the cluster assignments for easier processing, and then summarizes
        the content within each cluster.

        Parameters:
        - texts: A list of text documents to be processed.
        - level: An integer parameter that could define the depth or detail of processing.

        Returns:
        - Tuple containing two DataFrames:
          1. The first DataFrame (`df_clusters`) includes the original texts, their embeddings, and cluster assignments.
          2. The second DataFrame (`df_summary`) contains summaries for each cluster, the specified level of detail,
            and the cluster identifiers.
        """

        # Embed and cluster the texts, resulting in a DataFrame with 'text', 'embd', and 'cluster' columns
        df_clusters = embed_cluster_texts(texts)

        # Prepare to expand the DataFrame for easier manipulation of clusters
        expanded_list = []

        # Expand DataFrame entries to document-cluster pairings for straightforward processing
        for index, row in df_clusters.iterrows():
            for cluster in row["cluster"]:
                expanded_list.append(
                    {"text": row["text"], "embd": row["embd"], "cluster": cluster}
                )

        # Create a new DataFrame from the expanded list
        expanded_df = pd.DataFrame(expanded_list)

        # Retrieve unique cluster identifiers for processing
        all_clusters = expanded_df["cluster"].unique()

        print(f"--Generated {len(all_clusters)} clusters--")

        # Summarization
        template = """Here is a subset of a novel or an atricle.
        Give a detailed summary of the documentation provided.

        Documentation:
        {context}
        """
        prompt = ChatPromptTemplate.from_template(template)
        chain = prompt | model | StrOutputParser()

        # Format text within each cluster for summarization
        summaries = []
        for i in all_clusters:
            df_cluster = expanded_df[expanded_df["cluster"] == i]
            formatted_txt = fmt_txt(df_cluster)
            summaries.append(chain.invoke({"context": formatted_txt}))

        # Create a DataFrame to store summaries with their corresponding cluster and level
        df_summary = pd.DataFrame(
            {
                "summaries": summaries,
                "level": [level] * len(summaries),
                "cluster": list(all_clusters),
            }
        )

        return df_clusters, df_summary


    def recursive_embed_cluster_summarize(
        texts: List[str], level: int = 1, n_levels: int = 3
    ) -> Dict[int, Tuple[pd.DataFrame, pd.DataFrame]]:
        """
        Recursively embeds, clusters, and summarizes texts up to a specified level or until
        the number of unique clusters becomes 1, storing the results at each level.

        Parameters:
        - texts: List[str], texts to be processed.
        - level: int, current recursion level (starts at 1).
        - n_levels: int, maximum depth of recursion.

        Returns:
        - Dict[int, Tuple[pd.DataFrame, pd.DataFrame]], a dictionary where keys are the recursion
          levels and values are tuples containing the clusters DataFrame and summaries DataFrame at that level.
        """
        results = {}  # Dictionary to store results at each level

        # Perform embedding, clustering, and summarization for the current level
        df_clusters, df_summary = embed_cluster_summarize_texts(texts, level)

        # Store the results of the current level
        results[level] = (df_clusters, df_summary)

        # Determine if further recursion is possible and meaningful
        unique_clusters = df_summary["cluster"].nunique()
        if level < n_levels and unique_clusters > 1:
            # Use summaries as the input texts for the next level of recursion
            new_texts = df_summary["summaries"].tolist()
            next_level_results = recursive_embed_cluster_summarize(
                new_texts, level + 1, n_levels
            )

            # Merge the results from the next level into the current results dictionary
            results.update(next_level_results)

        return results

    # Execution starts here
    print("Starting the process")

    leaf_texts = docs_texts
    results = recursive_embed_cluster_summarize(leaf_texts, level=1, n_levels=3)

    all_texts = leaf_texts.copy()

    # Iterate through the results to extract summaries from each level and add them to all_texts
    for level in sorted(results.keys()):
        # Extract summaries from the current level's DataFrame
        summaries = results[level][1]["summaries"].tolist()
        # Extend all_texts with the summaries from the current level
        all_texts.extend(summaries)

    print(all_texts)

    print("Final Summaries extracted")




    # Example summaries and metadata
    # Generate embeddings
    embeddings = embd.encode(all_texts)

    print(len(embeddings[0]))
    # Prepare entities for insertion
    entities = [
        embeddings,  # Embedding vectors
        [book_title] * len(all_texts),  # Book titles
        all_texts  # Text chunks
    ]

    # Define the fields to match schema
    field_names = ["embedding", "book_title", "chunk"]

    # Insert into Milvus
    collection.insert(entities)
    collection.load()
    print("Data ingested into Milvus.")


collection.load()

def questionAnswer(query,book_title):

  embd = SentenceTransformer('all-MiniLM-L6-v2')


  def milvus_retriever(query, book_title, top_k=10):
      query_embedding = embd.encode([query]).tolist()
      search_params = {
          "metric_type": "COSINE",
          "params": {"nprobe": 10}
      }
      results = collection.search(query_embedding, "embedding", param=search_params, limit=top_k * 2, output_fields=["book_title", "chunk"])
      return results

  prompt = hub.pull("rlm/rag-prompt")

  def format_docs(docs):
      formatted_docs = []
      for result in docs:
        for doc in result:
            book_title = doc.entity.get("book_title")
            chunk = doc.entity.get("chunk")
            formatted_docs.append(f"{chunk} ")
      return "\n\n".join(formatted_docs)

  rag_chain = (
      {
          "context": lambda x: format_docs(milvus_retriever(x["query"], x["book_title"])),
          "question": lambda x: x["query"]
      }
      | prompt
      | model
      | StrOutputParser()
  )

  print(prompt)
  print(prompt.messages[0].prompt.template)

  response = rag_chain.invoke(
      {"query": query, "book_title": book_title}
  )
  print("Response received:")
  return response


def extract_text_from_pdf(pdf):
      pdf_reader = PdfReader(pdf)
      text = ""
      for page in pdf_reader.pages:
          text += page.extract_text()
      return text

def chunk_text(text, max_tokens=100):
        tokens = word_tokenize(text)
        chunks = [' '.join(tokens[i:i + max_tokens]) for i in range(0, len(tokens), max_tokens)]
        return chunks

pdf_path = "/content/Ansh Resume"

def main():

    # Extract text from the PDF
    text = extract_text_from_pdf(pdf_path)

    # Chunk the text
    chunks = chunk_text(text)

    # Generate summary
    summary = generateSummary(pdf_path,chunks)
    print("\nGenerated Summary:\n", summary)

    # Loop to handle user queries
    while True:
        query = input("\nEnter your query (or type 'exit' to quit): ")
        if query.lower() == 'exit':
            break
        response = questionAnswer(query, pdf_path)
        print("\nResponse:\n", response)

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

--Generated 1 clusters--


Llama.generate: prefix-match hit



        Human: Can you provide more details about the new insurance scheme for workers who died during the construction of a bridge mentioned in the text material?


llama_print_timings:        load time =  191675.21 ms
llama_print_timings:      sample time =      21.90 ms /    31 runs   (    0.71 ms per token,  1415.78 tokens per second)
llama_print_timings: prompt eval time =  941612.17 ms /  2388 tokens (  394.31 ms per token,     2.54 tokens per second)
llama_print_timings:        eval time =   21762.37 ms /    30 runs   (  725.41 ms per token,     1.38 tokens per second)
llama_print_timings:       total time =  963499.62 ms /  2418 tokens


['1 01st JAN , 2023 IMPORTANT DAYS Year 2023 being observed as International Year of millets • The year 2023 is being observed all over the world as the International Year of the millets . India is the largest producer of millets in the world . The Government of India proposed to the United Nations General Assembly to declare 2023 as the International Year of the Millets . • The proposal was accepted and passed by the 75th session of the UNGA in 2021 and 2023 was declared as the International Year of Millets . The Government of India aims', "to make the International Year of the millets a ‘ People ’ s Movement ’ and position India as the ‘ Global Hub for Millets ’ . INDIAN AFFAIRS India 's first underwater metro expected to operational by December 2023 • In a first , Kolkata will soon have India ’ s first underwater metro service . The Kolkata Metro Rail Corporation announced that the East -West Metro Corridor project is expected to be completed by December 2023 . The Kolkata metro tun

In [ ]:
!pip install  streamlit_extras streamlit pyngrok PyPDF2 nltk

In [ ]:

%%writefile app.py

import streamlit as st
from streamlit_extras.add_vertical   _space import add_vertical_space



# Background images add function
def add_bg_from_local(image_file):
    with open(image_file, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())
    st.markdown(
        f"""
        <style>
        .stApp {{
            background-image: url(data:image/{"jpeg"};base64,{encoded_string.decode()});
            background-size: cover;
        }}
        </style>
        """,
        unsafe_allow_html=True
    )

# Sidebar contents
with st.sidebar:
    st.title('LLM-LANGCHAIN CHATBOT')
    st.markdown('''
    ## About APP:

    The app's primary resource is utilised to create::

    - [streamlit](https://streamlit.io/)
    - [Langchain](https://docs.langchain.com/docs/)


    ## About me:

    - [Linkedin](https://www.linkedin.com/in/anshchawla1/)

    ''')

    st.write('Question Answer BOT ')

# Load environment variables
# load_dotenv()



def main():
    st.header("Chat with your pdf file")


    # Upload your PDF file
    pdf = st.file_uploader("Upload your PDF", type='pdf')

    if 'pdf_name' not in st.session_state:
            st.session_state.pdf_name = None
            st.session_state.summary = None
            st.session_state.text_chunks = None


    if pdf is not None:
        pdf_name = pdf.name

        if st.session_state.pdf_name != pdf_name:
            # Extract text from the PDF
            extracted_text = extract_text_from_pdf(pdf)

            st.session_state.pdf_name = pdf_name

            st.session_state.text_chunks = chunk_text(extracted_text, max_tokens=100)

            st.session_state.summary = generateSummary(st.session_state.text_chunks)

            st.write("**Extracted text from the PDF (first chunk):**")
            st.write(st.session_state.text_chunks[0] + '...')

        query = st.text_input("Ask questions about the content of your uploaded PDF")

        if query:
            # Placeholder for search and QA logic
            st.write(f"**Query:** {query}")
            response =  questionAnswer(query,pdf_name)
            st.write(f"**Response:** {response}")

if __name__ == "__main__":
    main()


Overwriting app.py


In [ ]:
!npm install localtunnel


up to date, audited 23 packages in 657ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.138.223.194


In [ ]:
!npx localtunnel --port 8501 --timeout 600


your url is: https://all-animals-pick.loca.lt
^C
